In [15]:
import gdal, osr
from skimage.graph import route_through_array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio

In [2]:
def raster2array(rasterfn):
    raster = gdal.Open(rasterfn)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    return array

In [3]:
def coord2pixelOffset(rasterfn, x, y):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    xOffset = int((x - originX)/ pixelWidth)
    yOffset = int((y - originY)/pixelHeight)
    return xOffset,yOffset

In [4]:
# Path
def createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord):   

    # coordinates to array index
    startCoordX = startCoord[0]
    startCoordY = startCoord[1]
    startIndexX,startIndexY = coord2pixelOffset(CostSurfacefn,startCoordX,startCoordY)

    stopCoordX = stopCoord[0]
    stopCoordY = stopCoord[1]
    stopIndexX,stopIndexY = coord2pixelOffset(CostSurfacefn,stopCoordX,stopCoordY)

    # create path
    indices, weight = route_through_array(costSurfaceArray, (startIndexY,startIndexX), (stopIndexY,stopIndexX),geometric=True,fully_connected=True)
    indices = np.array(indices).T
    path = np.zeros_like(costSurfaceArray)
    path[indices[0], indices[1]] = 1
    return path
        


In [5]:
#Least Cost Tool
def array2raster(newRasterfn,rasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3] 
    pixelWidth = geotransform[1] 
    pixelHeight = geotransform[5]
    cols = array.shape[1]
    rows = array.shape[0]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache() 

In [6]:

def main(CostSurfacefn,outputPathfn,startCoord,stopCoord):

    costSurfaceArray = raster2array(CostSurfacefn) # creates array from cost surface raster

    pathArray = createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord) # creates path array
 
    array2raster(outputPathfn,CostSurfacefn,pathArray) # converts path array to raster

In [2]:
CostSurface  = 'E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/1-NewMaster/THamStAll4Friction1.tif'

In [8]:
#Route Creation
if __name__ == "__main__":
    CostSurfacefn = CostSurface
    startCoord = (536552.739718,184199.345531)
    stopCoord = (533542.17254,180680.0646)
    outputPathfn = 'E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/1-NewMaster/test1.tif'
    main(CostSurfacefn,outputPathfn,startCoord,stopCoord)

In [16]:
StartpointsEndpoints = pd.read_csv('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/1-NewMaster/points/OriginDestiPoints.csv')
StartpointsEndpoints.head()
    

,FID,X,Y
0,0,536552.739718,184199.345531
1,1,535730.226088,182879.641548
2,2,534644.392062,180140.151694
3,3,537923.475729,182736.088233
4,4,537011.546346,179307.633403


In [17]:
for i in range(len(StartpointsEndpoints)):
    if __name__ == "__main__":
        CostSurfacefn = CostSurface
        startCoord = (StartpointsEndpoints.loc[i,'X'],StartpointsEndpoints.loc[i,'Y'])
        for j in range(len(StartpointsEndpoints)):
            stopCoord = (StartpointsEndpoints.loc[j,'X'],StartpointsEndpoints.loc[j,'Y'])
            outputPathfn = 'E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/1-NewMaster/output/path'+ str(i)+str(j)+'.tif'
            main(CostSurfacefn,outputPathfn,startCoord,stopCoord)


In [84]:
for i in range(3):
    for j in range(3):
        print("file_" + str(i)+str(j) + ".dat")

file_00.dat
file_01.dat
file_02.dat
file_10.dat
file_11.dat
file_12.dat
file_20.dat
file_21.dat
file_22.dat


In [29]:
!gdal_calc.py -A CostSurface -B CostSurface --outfile='E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/1-NewMaster/product.tif' --calc=A+B 

Traceback (most recent call last):
  File "C:\Users\Blusubmarine\Anaconda3\Scripts\gdal_calc.py", line 4, in <module>
    __import__('pkg_resources').run_script('GDAL==2.2.4', 'gdal_calc.py')
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 2927, in <module>
    @_call_aside
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 2913, in _call_aside
    f(*args, **kwargs)
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 2940, in _initialize_master_working_set
    working_set = WorkingSet._build_master()
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 635, in _build_master
    ws.require(__requires__)
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 943, in require
    needed = self.resolve(parse_requirements(requirements))
  File "C:\Python27\ArcGIS10.5\lib\site-packages\pkg_resources\__init__.py", line 829, in resolve
    raise

In [2]:
import pandas as pd
NO2Histogram = pd.read_csv('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/data/Histogram Table/HistogramNO2.csv')

In [16]:
#Test visualization of the NO2 Pollutants for routes using Histogram
NO2Histogram.head()
NO2Histogram = NO2Histogram.rename(columns={1:48})
NO2HistogramTest = pd.read_csv('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/data/Histogram Table/HistogramNO2_5.csv')
NO2HistogramTest.head()
plt.hist(NO2HistogramTest['NO2'])
plt.hist(NO2HistogramTest['NO2'], color = 'blue', edgecolor = 'black', bins = int(160/5))

sns.distplot(NO2HistogramTest['NO2'], hist=True, kde=False, bins=int(92/4), color = 'blue',hist_kws={'edgecolor':'black'})

In [68]:
#Data Normalization and Final Cost Surface Preparation
TowerNO2 = ('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/TowerNO2Rectangle.tif')
TowerNOx = ('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/TowerNOx.tif')
TowerPM10 = ('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/TowerPM10.tif')
TowerPM25 = ('E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/TowerPM25.tif')


with rasterio.open(TowerNO2) as TowerNO2:
    NO2 = TowerNO2.read()
    
with rasterio.open(TowerNOx) as TowerNOx:
    NOx = TowerNOx.read(1)
    
with rasterio.open(TowerPM10) as TowerPM10:
    PM10 = TowerPM10.read()
    
with rasterio.open(TowerPM25) as TowerPM25:
    PM25 = TowerPM25.read()
  
    

NO2Normal = (NO2 - NO2.min()) /(NO2.max() - NO2.min())

NO2Normalized = 'E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/NO2Normalized.tif'

with rasterio.open(NO2Normalized, 'w',**TowerNO2.meta) as output:
    output.write(NO2Normal)

Final = NO2Normal + NOxNormal + PM25Normal + PM10Normal   
    


C:\Users\Blusubmarine\Anaconda3\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


In [ ]:
#Analysing Maximum and Minimum Values of Pollutants
with rasterio.open('R.byte.tif') as src:
arr = src.read()
arr = (arr * 9) / 5 + 32



 myTestlayer = myTestlayer + 1000
    xMax = myTestlayer.max()
    xMin = myTestlayer.min()

xMax
xMin
    

with rasterio.open('output.tif', 'w',**src.meta) as dst:
    dst.write_band(1, arr[0])   
    
    
NO2Normalized = 'E:/2018/CASA/Term 2/4-Spatial Data Capture/0-coursework/pyhtonCoding/rasterCalculation/pollutants/NO2Normalized.tif'

with rasterio.open(NO2Normalized, 'w',**TowerNO2.meta) as output:
    output.write(NO2Normal)
    
NO2Normal = (NO2 - NO2.min()) /(NO2.max() - NO2.min())